<h1 style="text-align: center;"> PUTAMEN - MEAN INTENSITY EXTRACTION </h1>

The following code was used to extract mean signal intensities in putamen (left, right and both hemispheres)

In [ ]:
import os
import pandas as pd
import subprocess

In [ ]:
# define directories and putamen mask file names
base_dir = "/hus/home/oliwin/data/study_name"

mask_put_l = f"{base_dir}/putamen_L_mask.nii.gz"
mask_put_r = f"{base_dir}/putamen_R_mask.nii.gz"
putamen_mask = f"{base_dir}/putamen_mask.nii.gz"

# fusing left and right hemispheres together
!fslmaths {mask_put_l} -add {mask_put_r} {putamen_mask}

In [ ]:
# dictionary to store participant-wise data
patients_data = {}

# loop through subdirectories 'FSL' in base_dir (recursively)
for root, dirs, files in os.walk(base_dir):
    if "FSL" in dirs:
        fsl_dir = os.path.join(root, "FSL")
        
        # extract participant number from folder name 
        patient_number = os.path.basename(root)
        
        # loop through NIfTI files in the FSL directory
        for filename in os.listdir(fsl_dir):
            # process only files with "mni", "nii.gz"
            if (
                "mni" in filename
                and "mask" not in filename
                and "new" not in filename
                and "flirt" not in filename
                and "flirt_masked" not in filename
                and not filename.endswith(".mat")
                and filename.endswith(".nii.gz")
            ):
                input_file = os.path.join(fsl_dir, filename)

                
                print(f"Processing file: {input_file}")

                # loop through the putamen masks (left, right and combined) - each MRI file is processed three times
                for mask_file in [putamen_mask, mask_put_r, mask_put_l]:
                    try:
                        # use FLIRT to perform registration of the putamen masks
                        flirt_output = f"{input_file}_flirt_{os.path.basename(mask_file).split('.')[0]}"
                        flirt_matrix = f"{flirt_output}.mat"
                        flirt_command = f"flirt -in {input_file} -ref {mask_file} -out {flirt_output} -omat {flirt_matrix}"
                        process_flirt = subprocess.run(flirt_command, shell=True, capture_output=True, text=True)

                        if process_flirt.returncode != 0:
                            print(f"Error applying FLIRT to {filename} with mask {mask_file}: {process_flirt.stderr}")
                            continue  # continue with the next mask for same file

                        # use fslmaths to apply the masks and threshold negative values to zero
                        flirt_masked = f"{flirt_output}_masked"
                        fslmaths_command = f"fslmaths {flirt_output} -mas {mask_file} -thr 0 {flirt_masked}"
                        process_fslmaths = subprocess.run(fslmaths_command, shell=True, capture_output=True, text=True)

                        if process_fslmaths.returncode != 0:
                            print(f"Error applying fslmaths to {filename} with mask {mask_file}: {process_fslmaths.stderr}")
                            continue  # Continue with the next mask for same file

                        # use fslstats to extract mean intensity within the region
                        stats_command = f"fslstats {flirt_masked} -M"  # -M for Mean Intensity 
                        process_stats = subprocess.run(stats_command, shell=True, capture_output=True, text=True)

                        if process_stats.returncode != 0:
                            print(f"Error processing {filename}: {process_stats.stderr}")
                            continue  # continue with the next mask for same file

                       
                        stats = process_stats.stdout.strip().split()
                        mean_intensity = float(stats[0])  

                        # remove intermediate files and temporary NIfTI outputs
                        try:
                            for file_to_delete in [flirt_matrix, flirt_output + ".nii.gz", flirt_masked + ".nii.gz"]:
                                if os.path.exists(file_to_delete):
                                    os.remove(file_to_delete)
                                else:
                                    print(f"File not found: {file_to_delete}")
                        except OSError as e:
                            print(f"Error deleting files for {filename}: {e}")

                        # add the data to participant's entry
                        if patient_number not in patients_data:
                            patients_data[patient_number] = {}

                        mask_name = os.path.basename(mask_file).split('.')[0]  
                        filename_base = filename.replace('.nii.gz', '')  
                        mask_key = f'{mask_name}_{filename_base}'

                        patients_data[patient_number][mask_key] = mean_intensity

                    except Exception as e:
                        print(f"Error processing {filename} with mask {mask_file} for patient {patient_number}: {e}")
                        continue  # continue to the next mask or next file

# preparing the dataframe to store mean intensities
header = ['Patient Number']
all_keys = set()
for intensities in patients_data.values():
    all_keys.update(intensities.keys())
header.extend(sorted(all_keys)) 

output_data = []
for patient_number, intensities in patients_data.items():
    row = [patient_number]
    for col in header[1:]:
        row.append(intensities.get(col, None))  # use None for missing values
    output_data.append(row)

# creating the dataframe
output_df = pd.DataFrame(output_data, columns=header)

# saving the dataframe as csv for further analysis
output_file = 'mri_data_putamen.csv'
output_df.to_csv(output_file, index=False)
print(f"Transformed CSV saved to {output_file}")

Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_meSWIM_HPF_filled_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_meSWIM_filled_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_meSWIM_filled_MIP_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_mpSWIM_ECHO-3_e3_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_pSWIM_ECHO-3_e3_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_simCSF_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_simFLAIR_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_simGM_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_simWM_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/study_name/0000D71C/FSL/STAGE_sim_GRE_reg_mni.nii.gz
Processing file: /hus/home/oliwin/data/st